In [10]:
import glob
import json

import numpy as np
from PIL import Image

In [6]:
with open('../../../retina-datasets/diaretdb1_v1.1/diaretdb1_v_1_1/resources/traindatasets/trainset.txt') as file:
    train = file.readlines()

with open('../../../retina-datasets/diaretdb1_v1.1/diaretdb1_v_1_1/resources/testdatasets/testset.txt') as file:
    test = file.readlines()

In [8]:
train = [tmp.strip().split('.')[5:] for tmp in train]
test = [tmp.strip().split('.')[5:] for tmp in test]

In [11]:
mask = np.array(Image.open('../../../retina-datasets/diaretdb1_v1.1/diaretdb1_v_1_1/resources/images/ddb1_fundusmask/fmask.tif').getchannel(0))

In [37]:
gt_he = glob.glob('../../../retina-datasets/diaretdb1_v1.1/diaretdb1_v_1_1/resources/images/ddb1_groundtruth/hardexudates/*.png')
gt_se = glob.glob('../../../retina-datasets/diaretdb1_v1.1/diaretdb1_v_1_1/resources/images/ddb1_groundtruth/softexudates/*.png')
gt_rsd = glob.glob('../../../retina-datasets/diaretdb1_v1.1/diaretdb1_v_1_1/resources/images/ddb1_groundtruth/redsmalldots/*.png')
gt_hr = glob.glob('../../../retina-datasets/diaretdb1_v1.1/diaretdb1_v_1_1/resources/images/ddb1_groundtruth/hemorrhages/*.png')

In [39]:
distributions = {}

for image in gt_he:
    identifier = image.split('/')[-1].split('.')[0][5:]
    img = np.array(Image.open(image).getchannel(0))
    counts_no_fov = np.unique(img, return_counts=True)
    counts_fov = np.unique(img[mask], return_counts=True)

    if identifier not in distributions:
        distributions[identifier] = {'hardexudates': {'fov': {'values': counts_fov[0].tolist(), 'counts': counts_fov[1].tolist()},
                                                        'no_fov': {'values': counts_no_fov[0].tolist(), 'counts': counts_no_fov[1].tolist()}}}

for image in gt_se:
    identifier = image.split('/')[-1].split('.')[0][5:]
    img = np.array(Image.open(image).getchannel(0))
    counts_no_fov = np.unique(img, return_counts=True)
    counts_fov = np.unique(img[mask], return_counts=True)

    if identifier not in distributions:
        distributions[identifier]['softexudates'] = {'fov': {'values': counts_fov[0].tolist(), 'counts': counts_fov[1].tolist()},
                                                        'no_fov': {'values': counts_no_fov[0].tolist(), 'counts': counts_no_fov[1].tolist()}}

for image in gt_rsd:
    identifier = image.split('/')[-1].split('.')[0][5:]
    img = np.array(Image.open(image).getchannel(0))
    counts_no_fov = np.unique(img, return_counts=True)
    counts_fov = np.unique(img[mask], return_counts=True)

    if identifier not in distributions:
        distributions[identifier]['redsmalldots'] = {'fov': {'values': counts_fov[0].tolist(), 'counts': counts_fov[1].tolist()},
                                                        'no_fov': {'values': counts_no_fov[0].tolist(), 'counts': counts_no_fov[1].tolist()}}

for image in gt_hr:
    identifier = image.split('/')[-1].split('.')[0][5:]
    img = np.array(Image.open(image).getchannel(0))
    counts_no_fov = np.unique(img, return_counts=True)
    counts_fov = np.unique(img[mask], return_counts=True)

    if identifier not in distributions:
        distributions[identifier]['hemorrhages'] = {'fov': {'values': counts_fov[0].tolist(), 'counts': counts_fov[1].tolist()},
                                                        'no_fov': {'values': counts_no_fov[0].tolist(), 'counts': counts_no_fov[1].tolist()}}


In [40]:
distributions

{'079': {'hardexudates': {'fov': {'values': [0], 'counts': [1407235]},
   'no_fov': {'values': [0], 'counts': [1728000]}}},
 '026': {'hardexudates': {'fov': {'values': [0, 42, 63, 105, 126, 168, 189],
    'counts': [1362069, 24212, 10743, 4863, 5183, 66, 99]},
   'no_fov': {'values': [0, 42, 63, 105, 126, 168, 189],
    'counts': [1682834, 24212, 10743, 4863, 5183, 66, 99]}}},
 '017': {'hardexudates': {'fov': {'values': [0,
     42,
     63,
     84,
     105,
     126,
     189,
     252],
    'counts': [1252922, 25173, 54751, 6, 1541, 22461, 12478, 37903]},
   'no_fov': {'values': [0, 42, 63, 84, 105, 126, 189, 252],
    'counts': [1573687, 25173, 54751, 6, 1541, 22461, 12478, 37903]}}},
 '065': {'hardexudates': {'fov': {'values': [0], 'counts': [1407235]},
   'no_fov': {'values': [0], 'counts': [1728000]}}},
 '073': {'hardexudates': {'fov': {'values': [0], 'counts': [1407235]},
   'no_fov': {'values': [0], 'counts': [1728000]}}},
 '060': {'hardexudates': {'fov': {'values': [0], 'cou

In [41]:
distributions_train = {key: value for key, value in distributions.items() if key in train}
distributions_test = {key: value for key, value in distributions.items() if key in test}

In [44]:
citation = """
@inproceedings{Kauppi2007TheDD,
  title={The DIARETDB1 Diabetic Retinopathy Database and Evaluation Protocol},
  author={Tomi Kauppi and Valentina Kalesnykiene and J. K{\"a}m{\"a}r{\"a}inen and Lasse Lensu and Iiris Sorri and A. Raninen and R. Voutilainen and Hannu Uusitalo and Heikki K{\"a}lvi{\"a}inen and Juhani Pietil{\"a}},
  booktitle={British Machine Vision Conference},
  year={2007},
  url={https://api.semanticscholar.org/CorpusID:15483141}
}"""

In [45]:
with open('diaretdb1_train.json', 'w') as file:
    json.dump({'distribution': distributions_train, 'citation': citation}, file)

with open('diaretdb1_test.json', 'w') as file:
    json.dump({'distribution': distributions_test, 'citation': citation}, file)